# MedPix Data Cleaning and Preprocessing
## Preparing Cases for the Web Application

**Objective**: Clean and structure the raw MedPix Cases.json data for the frontend application

**Tasks**:
1. Load and inspect raw data
2. Extract and clean key fields
3. Create structured case objects
4. Save processed data for API consumption

In [1]:
# Import required libraries
import json
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter

print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


## 1. Load Raw Data

In [2]:
# Load the raw Cases.json file
data_path = Path('../data/archive/Cases.json')

with open(data_path, 'r', encoding='utf-8') as f:
    raw_cases = json.load(f)

print(f"📊 Total cases loaded: {len(raw_cases)}")
print(f"\n📝 Sample case structure:")
print(f"Keys: {list(raw_cases[0].keys())}")

📊 Total cases loaded: 7432

📝 Sample case structure:
Keys: ['URL', 'Case Folder', 'Image Paths', 'Case Title']


## 2. Inspect Data Structure

In [3]:
# Examine first case in detail
sample_case = raw_cases[0]

print("🔍 Sample Case Fields:")
print("="*50)
for key, value in sample_case.items():
    if isinstance(value, str):
        preview = value[:100] + "..." if len(value) > 100 else value
    elif isinstance(value, list):
        preview = f"List with {len(value)} items"
    else:
        preview = value
    print(f"{key}: {preview}\n")

🔍 Sample Case Fields:
URL: https://medpix.nlm.nih.gov/bycase?id=f1c6eccd-a9e9-4ca3-beb6-30c25d763ecc

Case Folder: medpix_data_final\case_8892378009084536600

Image Paths: List with 23 items

Case Title: CASE
A Neck And Wrist Pain: Bilateral Carpal Tunnel Syndrome, Cervical Subluxation Knee Pain: Post-T...



## 3. Define Data Cleaning Functions

In [4]:
def extract_case_id(case_folder):
    """Extract case ID from folder path"""
    match = re.search(r'case_(-?\d+)', case_folder)
    return match.group(1) if match else None

def clean_text(text):
    """Clean and normalize text fields"""
    if not isinstance(text, str):
        return ""
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters but keep basic punctuation
    text = text.strip()
    return text

def extract_diagnosis(case_title):
    """Extract diagnosis from case title"""
    if not isinstance(case_title, str):
        return "Unknown"
    
    # Case title often contains diagnosis after "CASE" keyword
    # Try to extract the first meaningful part
    lines = case_title.split('\n')
    if len(lines) > 1:
        diagnosis = lines[1].strip()
        return clean_text(diagnosis) if diagnosis else "Unknown"
    return clean_text(case_title[:200])

def count_images(image_paths):
    """Count number of images for a case"""
    return len(image_paths) if isinstance(image_paths, list) else 0

def fix_image_paths(image_paths):
    """Convert Windows paths to Unix paths"""
    if not isinstance(image_paths, list):
        return []
    # Replace backslashes with forward slashes
    return [path.replace('\\', '/') for path in image_paths]

print("✅ Cleaning functions defined!")

✅ Cleaning functions defined!


## 4. Process All Cases

In [5]:
def process_case(raw_case, index):
    """Process a single case into clean structure"""
    
    # Extract case ID
    case_id = extract_case_id(raw_case.get('Case Folder', ''))
    if not case_id:
        case_id = f"case_{index}"
    
    # Extract diagnosis from title
    diagnosis = extract_diagnosis(raw_case.get('Case Title', ''))
    
    # Clean image paths
    image_paths = fix_image_paths(raw_case.get('Image Paths', []))
    
    # Create clean case object
    clean_case = {
        'id': case_id,
        'url': raw_case.get('URL', ''),
        'diagnosis': diagnosis,
        'caseTitle': clean_text(raw_case.get('Case Title', '')),
        'history': clean_text(raw_case.get('History', '')),
        'exam': clean_text(raw_case.get('Exam', '')),
        'findings': clean_text(raw_case.get('Findings', '')),
        'caseDiagnosis': clean_text(raw_case.get('Case Diagnosis', '')),
        'treatment': clean_text(raw_case.get('Treatment & Follow Up', '')),
        'discussion': clean_text(raw_case.get('Discussion', '')),
        'differentialDiagnosis': clean_text(raw_case.get('Differential Diagnosis', '')),
        'imageCount': len(image_paths),
        'imagePaths': image_paths[:5],  # Keep only first 5 images for preview
        'caseFolder': raw_case.get('Case Folder', '').replace('\\', '/')
    }
    
    return clean_case

# Process all cases
print("🔄 Processing cases...")
processed_cases = []

for idx, raw_case in enumerate(raw_cases):
    try:
        clean_case = process_case(raw_case, idx)
        processed_cases.append(clean_case)
    except Exception as e:
        print(f"⚠️ Error processing case {idx}: {e}")
        continue

print(f"✅ Successfully processed {len(processed_cases)} cases!")
print(f"⚠️ Failed to process {len(raw_cases) - len(processed_cases)} cases")

🔄 Processing cases...
✅ Successfully processed 7432 cases!
⚠️ Failed to process 0 cases


## 5. Data Quality Check

In [6]:
# Check data quality
df = pd.DataFrame(processed_cases)

print("📊 Processed Data Summary:")
print("="*50)
print(f"Total cases: {len(df)}")
print(f"\nData completeness:")
print(df.notna().sum())
print(f"\nImage statistics:")
print(df['imageCount'].describe())
print(f"\nSample diagnoses:")
print(df['diagnosis'].value_counts().head(10))

📊 Processed Data Summary:
Total cases: 7432

Data completeness:
id                       7432
url                      7432
diagnosis                7432
caseTitle                7432
history                  7432
exam                     7432
findings                 7432
caseDiagnosis            7432
treatment                7432
discussion               7432
differentialDiagnosis    7432
imageCount               7432
imagePaths               7432
caseFolder               7432
dtype: int64

Image statistics:
count    7432.000000
mean        5.503229
std         5.912677
min         0.000000
25%         2.000000
50%         4.000000
75%         7.000000
max       184.000000
Name: imageCount, dtype: float64

Sample diagnoses:
diagnosis
Osteochondroma               84
Acute Appendicitis           32
Osteoid Osteoma              32
Meningioma                   30
Osteosarcoma                 28
Osteochondritis Dissecans    28
Osteomyelitis                28
Bicornuate Uterus            2

## 6. Save Processed Data

In [7]:
# Create output directory if it doesn't exist
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save full processed cases
output_file = output_dir / 'cases_cleaned.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(processed_cases, f, indent=2, ensure_ascii=False)

print(f"✅ Saved {len(processed_cases)} cases to {output_file}")

# Also create a summary file with just essential fields for faster loading
cases_summary = []
for case in processed_cases:
    summary = {
        'id': case['id'],
        'diagnosis': case['diagnosis'],
        'imageCount': case['imageCount'],
        'thumbnail': case['imagePaths'][0] if case['imagePaths'] else None,
        'url': case['url']
    }
    cases_summary.append(summary)

summary_file = output_dir / 'cases_summary.json'
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(cases_summary, f, indent=2, ensure_ascii=False)

print(f"✅ Saved summary to {summary_file}")

✅ Saved 7432 cases to ../data/processed/cases_cleaned.json
✅ Saved summary to ../data/processed/cases_summary.json


## 7. Inspect Processed Data

In [8]:
# Show sample of processed case
print("📋 Sample Processed Case:")
print("="*50)
print(json.dumps(processed_cases[0], indent=2))

📋 Sample Processed Case:
{
  "id": "8892378009084536600",
  "url": "https://medpix.nlm.nih.gov/bycase?id=f1c6eccd-a9e9-4ca3-beb6-30c25d763ecc",
  "diagnosis": "A Neck And Wrist Pain: Bilateral Carpal Tunnel Syndrome, Cervical Subluxation Knee Pain: Post-Traumatic Osteoarthritis, Chronic ACL & PCL Tears, Torn Lateral Meniscus Back Pain: Traumatic Herniated Discs At L4-L5 And L5-S1 With Lumbar Radiculopathy B Patient B Was Diagnosed With A Compression Fracture In The Vertebral Body Of L2, Greater Trochanteric Bursitis With Tendinopathy Of The Inserting Gluteus Medius Tendon, And A Bone Contusion Of The Tibia.",
  "caseTitle": "CASE A Neck And Wrist Pain: Bilateral Carpal Tunnel Syndrome, Cervical Subluxation Knee Pain: Post-Traumatic Osteoarthritis, Chronic ACL & PCL Tears, Torn Lateral Meniscus Back Pain: Traumatic Herniated Discs At L4-L5 And L5-S1 With Lumbar Radiculopathy B Patient B Was Diagnosed With A Compression Fracture In The Vertebral Body Of L2, Greater Trochanteric Bursitis 

## Summary

✅ **Data Processing Complete!**

- Loaded raw Cases.json
- Cleaned and structured all cases
- Fixed image paths (Windows → Unix)
- Extracted key fields (diagnosis, history, findings, etc.)
- Saved processed data to `data/processed/`

**Output Files:**
1. `cases_cleaned.json` - Full case data with all fields
2. `cases_summary.json` - Lightweight summary for gallery view

**Next Steps:**
- Create FastAPI backend to serve this data
- Build Cases Gallery in frontend
- Add image loading and display
- Implement search and filtering